<a href="https://colab.research.google.com/github/gkroger98/AI-Powered-Sales-Presentation-Builder/blob/main/AI_PowerPoint_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## api_key = ""
## model_name = "gpt-4"
## openai.api_key = api_key

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.9/680.9 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
!pip install openai==0.28 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
# openaimigrate

In [ ]:
!pip show openai

Name: openai
Version: 1.78.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
!pip install python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 12.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

# Define a test message to send to the API
test_message = "Hello, who are you?"

# Send the request using the new client interface
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": test_message}
    ],
    temperature=0.7
)

# Print the content of the response
print(response.choices[0].message.content)


I'm OpenAI's digital assistant. I'm here to help answer questions and provide useful information. How can I assist you today?


We’ll build the flow in sections, each 10–20 slides:
Section 1: Hook & Setup (20–30 slides)
Section 2: Proof of Concept / Trade Examples (100+ slides)
Section 3: Testimonials + Social Proof (20–30 slides)
Section 4: Education & Breakdowns (50+ slides)
Section 5: System Reveal & Offer Stack (40–50 slides)
Section 6: Urgency & Final CTA (5–10 slides)
Section 7: Bonus & FAQs (optional, 10–20)
Each section can be generated in batches of 20 slides, while maintaining shared session variables (like the chosen product, speaker name, past trades, etc.) to preserve continuity. I'm still looking into automating the conversion of the text to slides steps. I'll have to research what APIs are out there, how much they cost, cost to performance (skewing towards lesser cost for POC, and then we can upgrade for quality) all that good stuff. But yeah, working on it.

In [ ]:
## Slide Generation Code
## This script uses GPT to generate slides and saves them into JSON (inside folders like slide_outputs/slides_001_005.json, etc.).
## Keep this script separate. Example filename:
## generate_slides.py







import os
import json
import time
import logging
from openai import OpenAI

# --- Config --- #
OPENAI_API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
client = OpenAI(api_key=OPENAI_API_KEY)

PRODUCT_NAME = "SuperWidget X"
PRODUCT_CATEGORY = "B2B SaaS Platform"
TARGET_AUDIENCE = "IT Directors at mid-size enterprises"
TOTAL_SLIDES = 500
CHUNK_SIZE = 5
MODEL = "gpt-4"
OUTPUT_FOLDER = "slide_outputs"
SLIDE_LIMIT = 20  # Limit slide generation for testing or control

# --- Slide Structure Map --- #
SLIDE_STRUCTURE = [
    (1, 50, "Introduction"),
    (51, 125, "Customer Pain Points"),
    (126, 200, "Product Benefits"),
    (201, 300, "Product Deep Dive"),
    (301, 400, "Use Cases & Testimonials"),
    (401, 475, "Closing & Urgency"),
    (476, 500, "Final Call-to-Action")
]

SLIDE_SECTION_MAP = {start: section for start, end, section in SLIDE_STRUCTURE}

def get_section_name(slide_number):
    for start in sorted(SLIDE_SECTION_MAP.keys(), reverse=True):
        if slide_number >= start:
            return SLIDE_SECTION_MAP[start]
    return "Unknown Section"

def build_prompt(start, end, section):
    # Now request output as JSON for easier parsing downstream
    return f"""
You are a top-performing charismatic, persuasive salesperson copywriter creating content for a high-impact, energetic and urgent pitch deck.

Audience: {TARGET_AUDIENCE}
Product: {PRODUCT_NAME} ({PRODUCT_CATEGORY})
Your job is to write persuasive copy for slides {start} through {end} in the {section} section of a 500-slide sales pitch.

Follow these sales principles:
1. Customer is always right
2. Focus on benefits, not features
3. Build urgency and authority
4. Close hard in the final sections

Maintain continuity with previous slides and tone. Use compelling language that keeps the energy high. If appropriate, insert:
- <CHART_PLACEHOLDER>
- <TABLE_PLACEHOLDER>
- <TESTIMONIAL_PLACEHOLDER>
- <SPEAKER_NOTE>

Each slide should be about 100–150 words.

Please output a JSON array where each element is an object representing one slide, with these keys:
- "slide_number" (int)
- "title" (string, optional)
- "content" (string)

Example output format:

[
  {{
    "slide_number": {start},
    "title": "Slide title here",
    "content": "Persuasive slide content here."
  }},
  ...
]

Begin:
""".strip()

def parse_slide_response(content):
    """
    Attempt to parse the GPT JSON response into Python objects.
    If parsing fails, log error and raise.
    """
    try:
        slides = json.loads(content)
        if not isinstance(slides, list):
            raise ValueError("Response JSON is not a list")
        return slides
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse GPT JSON response: {e}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error parsing response: {e}")
        raise

def make_request_with_retry(prompt, retries=3, delay=5):
    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.85,
                max_tokens=1500
            )
            return response
        except Exception as e:
            logging.error(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                raise

def generate_slides(limit=TOTAL_SLIDES):
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    for start in range(1, min(limit, TOTAL_SLIDES) + 1, CHUNK_SIZE):
        end = min(start + CHUNK_SIZE - 1, limit)
        section = get_section_name(start)
        prompt = build_prompt(start, end, section)

        try:
            logging.info(f"Generating slides {start}-{end} in section: {section}...")
            response = make_request_with_retry(prompt)

            content = response.choices[0].message.content.strip()
            slides = parse_slide_response(content)

            subfolder = os.path.join(OUTPUT_FOLDER, f"slides_{start}_{end}")
            os.makedirs(subfolder, exist_ok=True)

            json_path = os.path.join(subfolder, "slides.json")
            with open(json_path, "w", encoding="utf-8") as json_file:
                json.dump({
                    "start_slide": start,
                    "end_slide": end,
                    "section": section,
                    "slides": slides
                }, json_file, indent=2)

            txt_path = os.path.join(subfolder, "slides.txt")
            with open(txt_path, "w", encoding="utf-8") as txt_file:
                txt_file.write(f"--- Slides {start}-{end} ({section}) ---\n\n")
                for slide in slides:
                    txt_file.write(f"Slide {slide['slide_number']}:\n{slide.get('title', '')}\n{slide['content']}\n\n")

            time.sleep(1)

        except Exception as e:
            logging.error(f"Error generating slides {start}-{end}: {e}")
            continue

    logging.info(f"\n✅ Slide groups saved in '{OUTPUT_FOLDER}' folder")

# --- Logging Setup --- #
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Run it --- #
if __name__ == "__main__":
    generate_slides(limit=SLIDE_LIMIT)


# COPY OF ABOVE WITH SPECIFIC PROMPT

In [ ]:
## PowerPoint Generation Code (the new script I gave you)
## This script loads your saved slide content and turns it into a .pptx file using python-pptx.
## Save this as a separate script. Example filename:
## build_powerpoint.py


# build_powerpoint.py

import os
import json
import logging
from collections import defaultdict
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor

# --- Configurable Variables --- #
SLIDE_FOLDER = "slide_outputs"
OUTPUT_PATH = "SuperWidgetX_PitchDeck.pptx"
TITLE_FONT_SIZE = Pt(32)
BODY_FONT_SIZE = Pt(20)
MAX_RETRIES = 3
CHECKPOINT_FILE = "checkpoint.json"
BRAND_COLOR = RGBColor(0, 51, 102)
SECTION_ORDER = ["hook", "problem", "solution", "demo", "proof", "close"]

# --- Logging Setup --- #
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Placeholder Replacement --- #
def replace_placeholders(text, product_name, target_audience, product_category):
    replacements = {
        "<PRODUCT>": product_name,
        "<TARGET_AUDIENCE>": target_audience,
        "<PRODUCT_CATEGORY>": product_category,
    }
    for placeholder, replacement in replacements.items():
        text = text.replace(placeholder, replacement)
    return text

# --- Load Slide Data --- #
def load_slide_data():
    all_slide_data = defaultdict(list)
    logger.debug("🔍 Current Working Directory: %s", os.getcwd())

    if not os.path.exists(SLIDE_FOLDER):
        logger.warning("⚠️ '%s' directory not found. Creating it now...", SLIDE_FOLDER)
        os.makedirs(SLIDE_FOLDER)
        return all_slide_data

    logger.debug("📂 '%s' contents: %s", SLIDE_FOLDER, os.listdir(SLIDE_FOLDER))

    for subdir in os.listdir(SLIDE_FOLDER):
        full_subdir_path = os.path.join(SLIDE_FOLDER, subdir)
        if not os.path.isdir(full_subdir_path):
            logger.warning("⚠️ Skipping non-directory: %s", subdir)
            continue

        for file in os.listdir(full_subdir_path):
            if not file.endswith(".json"):
                logger.warning("⚠️ Skipping non-JSON file: %s", file)
                continue

            full_path = os.path.join(full_subdir_path, file)
            try:
                with open(full_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    if isinstance(data, dict) and "slides" in data:
                        slide_list = data["slides"]
                    elif isinstance(data, dict) and "title" in data and "content" in data:
                        slide_list = [data]
                    elif isinstance(data, list):
                        slide_list = data
                    else:
                        logger.warning("⚠️ Unrecognized slide format in %s — skipping", file)
                        continue

                    slide_list = [slide for slide in slide_list if "title" in slide and "content" in slide]
                    logger.debug("✅ Loaded %d valid slides from %s/%s", len(slide_list), subdir, file)
                    all_slide_data[subdir].extend(slide_list)

            except Exception as e:
                logger.exception("🔥 Failed to load %s: %s", file, e)

    return all_slide_data

# --- Checkpoint Handling --- #
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r") as f:
            logger.info(f"🔄 Loading checkpoint from {CHECKPOINT_FILE}...")
            return json.load(f)
    return {}

def save_checkpoint(processed_slides):
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump(processed_slides, f)

# --- Add Slide Function --- #
def add_slide(prs, title_text, body_text, product_name, target_audience, product_category):
    title_text = replace_placeholders(title_text, product_name, target_audience, product_category)
    body_text = replace_placeholders(body_text, product_name, target_audience, product_category)

    blank_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_layout)

    # Title
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9), Inches(1))
    title_frame = title_box.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = TITLE_FONT_SIZE
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].font.color.rgb = BRAND_COLOR

    # Body
    body_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.5), Inches(9), Inches(6))
    body_frame = body_box.text_frame
    body_frame.clear()

    for paragraph in body_text.split("\n"):
        if paragraph.strip():
            p = body_frame.add_paragraph()
            p.text = paragraph.strip()
            p.font.size = BODY_FONT_SIZE
            p.font.color.rgb = RGBColor(0, 0, 0)

# --- Process Individual Slide --- #
def process_slide(prs, slide, product_name, target_audience, product_category, retries=0):
    try:
        slide_number = slide.get("slide_number", "Unknown")
        title = slide.get("title", f"Slide {slide_number}")
        body = slide.get("content", "")

        if not title or not body:
            logger.warning(f"⚠️ Skipping empty slide {slide_number} — missing title or content.")
            return False

        add_slide(prs, title, body, product_name, target_audience, product_category)
        return True
    except Exception as e:
        logger.error(f"❌ Error processing slide {slide.get('slide_number', 'Unknown')}: {e}")
        if retries < MAX_RETRIES:
            return process_slide(prs, slide, product_name, target_audience, product_category, retries + 1)
        else:
            return False

# --- Main Generation Function --- #
def generate_powerpoint(product_name, target_audience, product_category, max_slides=20):
    prs = Presentation()
    grouped_slides = load_slide_data()
    processed_slides = load_checkpoint()
    slide_count = 0

    # Sort the sections by SECTION_ORDER
    sorted_section_keys = sorted(grouped_slides.keys(), key=lambda x: SECTION_ORDER.index(x) if x in SECTION_ORDER else len(SECTION_ORDER))

    for section in sorted_section_keys:
        slides = grouped_slides[section]
        sorted_slides = sorted(slides, key=lambda x: x.get("slide_number", 0))

        for slide in sorted_slides:
            slide_number = str(slide.get("slide_number", "Unknown"))
            if slide_number in processed_slides:
                logger.info(f"⏩ Skipping slide {slide_number} (already processed).")
                continue

            if slide_count >= max_slides:
                break

            success = process_slide(prs, slide, product_name, target_audience, product_category)
            if success:
                processed_slides[slide_number] = True
                save_checkpoint(processed_slides)
                slide_count += 1

    if slide_count == 0:
        logger.warning("⚠️ No valid slides were added to the presentation.")
    else:
        prs.save(OUTPUT_PATH)
        logger.info(f"🎉 PowerPoint saved to: {OUTPUT_PATH} with {slide_count} slides.")

# --- Manual Run --- #
if __name__ == "__main__":
    generate_powerpoint(
        product_name="SuperWidget X",
        target_audience="IT Directors at mid-size enterprises",
        product_category="B2B SaaS Platform",
        max_slides=20
    )


In [ ]:
## PowerPoint Generation Code (the new script I gave you)
## This script loads your saved slide content and turns it into a .pptx file using python-pptx.
## Save this as a separate script. Example filename:
## build_powerpoint.py


# build_powerpoint.py

import os
import json
import logging
from collections import defaultdict
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor

# --- Configurable Variables --- #
SLIDE_FOLDER = "slide_outputs"
OUTPUT_PATH = "SuperWidgetX_PitchDeck.pptx"
TITLE_FONT_SIZE = Pt(32)
BODY_FONT_SIZE = Pt(20)
MAX_RETRIES = 3
CHECKPOINT_FILE = "checkpoint.json"
BRAND_COLOR = RGBColor(0, 51, 102)
SECTION_ORDER = ["hook", "problem", "solution", "demo", "proof", "close"]

# --- Logging Setup --- #
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Placeholder Replacement --- #
def replace_placeholders(text, product_name, target_audience, product_category):
    replacements = {
        "<PRODUCT>": product_name,
        "<TARGET_AUDIENCE>": target_audience,
        "<PRODUCT_CATEGORY>": product_category,
    }
    for placeholder, replacement in replacements.items():
        text = text.replace(placeholder, replacement)
    return text

# --- Load Slide Data --- #
def load_slide_data():
    all_slide_data = defaultdict(list)
    logger.debug("🔍 Current Working Directory: %s", os.getcwd())

    if not os.path.exists(SLIDE_FOLDER):
        logger.warning("⚠️ '%s' directory not found. Creating it now...", SLIDE_FOLDER)
        os.makedirs(SLIDE_FOLDER)
        return all_slide_data

    logger.debug("📂 '%s' contents: %s", SLIDE_FOLDER, os.listdir(SLIDE_FOLDER))

    for subdir in os.listdir(SLIDE_FOLDER):
        full_subdir_path = os.path.join(SLIDE_FOLDER, subdir)
        if not os.path.isdir(full_subdir_path):
            logger.warning("⚠️ Skipping non-directory: %s", subdir)
            continue

        for file in os.listdir(full_subdir_path):
            if not file.endswith(".json"):
                logger.warning("⚠️ Skipping non-JSON file: %s", file)
                continue

            full_path = os.path.join(full_subdir_path, file)
            try:
                with open(full_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    if isinstance(data, dict) and "slides" in data:
                        slide_list = data["slides"]
                    elif isinstance(data, dict) and "title" in data and "content" in data:
                        slide_list = [data]
                    elif isinstance(data, list):
                        slide_list = data
                    else:
                        logger.warning("⚠️ Unrecognized slide format in %s — skipping", file)
                        continue

                    slide_list = [slide for slide in slide_list if "title" in slide and "content" in slide]
                    logger.debug("✅ Loaded %d valid slides from %s/%s", len(slide_list), subdir, file)
                    all_slide_data[subdir].extend(slide_list)

            except Exception as e:
                logger.exception("🔥 Failed to load %s: %s", file, e)

    return all_slide_data

# --- Checkpoint Handling --- #
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r") as f:
            logger.info(f"🔄 Loading checkpoint from {CHECKPOINT_FILE}...")
            return json.load(f)
    return {}

def save_checkpoint(processed_slides):
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump(processed_slides, f)

# --- Add Slide Function --- #
def add_slide(prs, title_text, body_text, product_name, target_audience, product_category):
    title_text = replace_placeholders(title_text, product_name, target_audience, product_category)
    body_text = replace_placeholders(body_text, product_name, target_audience, product_category)

    blank_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_layout)

    # Title
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9), Inches(1))
    title_frame = title_box.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = TITLE_FONT_SIZE
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].font.color.rgb = BRAND_COLOR

    # Body
    body_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.5), Inches(9), Inches(6))
    body_frame = body_box.text_frame
    body_frame.clear()

    for paragraph in body_text.split("\n"):
        if paragraph.strip():
            p = body_frame.add_paragraph()
            p.text = paragraph.strip()
            p.font.size = BODY_FONT_SIZE
            p.font.color.rgb = RGBColor(0, 0, 0)

# --- Process Individual Slide --- #
def process_slide(prs, slide, product_name, target_audience, product_category, retries=0):
    try:
        slide_number = slide.get("slide_number", "Unknown")
        title = slide.get("title", f"Slide {slide_number}")
        body = slide.get("content", "")

        if not title or not body:
            logger.warning(f"⚠️ Skipping empty slide {slide_number} — missing title or content.")
            return False

        add_slide(prs, title, body, product_name, target_audience, product_category)
        return True
    except Exception as e:
        logger.error(f"❌ Error processing slide {slide.get('slide_number', 'Unknown')}: {e}")
        if retries < MAX_RETRIES:
            return process_slide(prs, slide, product_name, target_audience, product_category, retries + 1)
        else:
            return False

# --- Main Generation Function --- #
def generate_powerpoint(product_name, target_audience, product_category, max_slides=20):
    prs = Presentation()
    grouped_slides = load_slide_data()
    processed_slides = load_checkpoint()
    slide_count = 0

    # Sort the sections by SECTION_ORDER
    sorted_section_keys = sorted(grouped_slides.keys(), key=lambda x: SECTION_ORDER.index(x) if x in SECTION_ORDER else len(SECTION_ORDER))

    for section in sorted_section_keys:
        slides = grouped_slides[section]
        sorted_slides = sorted(slides, key=lambda x: x.get("slide_number", 0))

        for slide in sorted_slides:
            slide_number = str(slide.get("slide_number", "Unknown"))
            if slide_number in processed_slides:
                logger.info(f"⏩ Skipping slide {slide_number} (already processed).")
                continue

            if slide_count >= max_slides:
                break

            success = process_slide(prs, slide, product_name, target_audience, product_category)
            if success:
                processed_slides[slide_number] = True
                save_checkpoint(processed_slides)
                slide_count += 1

    if slide_count == 0:
        logger.warning("⚠️ No valid slides were added to the presentation.")
    else:
        prs.save(OUTPUT_PATH)
        logger.info(f"🎉 PowerPoint saved to: {OUTPUT_PATH} with {slide_count} slides.")

# --- Manual Run --- #
if __name__ == "__main__":
    generate_powerpoint(
        product_name="SuperWidget X",
        target_audience="IT Directors at mid-size enterprises",
        product_category="B2B SaaS Platform",
        max_slides=20
    )


In [ ]:
#clean out the slide_outputs folder
import shutil
import os

# Path to the slide_outputs folder
slide_outputs_folder = 'slide_outputs'

# Check if the folder exists and remove it if it does
if os.path.exists(slide_outputs_folder):
    shutil.rmtree(slide_outputs_folder)
    print(f"Successfully deleted the {slide_outputs_folder} folder and all its contents.")
else:
    print(f"The {slide_outputs_folder} folder does not exist.")


Successfully deleted the slide_outputs folder and all its contents.
